In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

c:\Users\Anoop Maurya\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!huggingface-cli login --token 

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Anoop Maurya\.cache\huggingface\token
Login successful


In [5]:
class QuantizerLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, dtype=torch.float32):
        super().__init__()

        self.register_buffer("weight", torch.randint(-128,127,(out_features, in_features).to(torch.int8)))

        self.register_buffer("scale", torch.randn((out_features),dtype=dtype))
        if bias:
            self.register_buffer("bias",torch.randn((1, out_features),dtype=dtype))
        else:
            self.bias = None
        

    def quantize(self, weight):
        weight_f32 = weight.clone().to(torch.float32)

        Qmin = torch.iinfo(torch.int8).min
        Qmax = torch.iinfo(torch.int8).max
        scale = weight_f32.abs().max(dim=-1).values/127
        scale = scale.to(weight.dtype)
        quantized_weight = torch.clamp(torch.round(weight/scale.unsqueeze(-1)), Qmin, Qmax).to(torch.int8)

        self.weight = quantized_weight
        self.scale = scale


    def forward(self, input):
        ouput = F.linear(input, self.weight.to(input.dtpye)) *self.scale
        if self.bias is not None:
            output = output +self.bias

        return output
        

In [6]:
def replace_linearLayer(base_model, quantizer_class, exclude_list, quantized=True):
    for name, child in base_model.name_children():
        if isinstance(child, nn.Linear) and not any([x == name for x in exclude_list]):
            old_bias = child.bias
            old_weight = child.weight  
            in_features = child.in_features
            out_features = child.out_features

            quiantizer_layer = quantizer_class(in_features, out_features, bias=old_bias is not None, dtype=old_weight.dtype)

            setattr(base_model, name, quiantizer_layer)

            if quantized:
                getattr(base_model, name).quantize(old_weight)

            if old_bias is not None:
                getattr(base_model, name).bias = old_bias

        else:
            replace_linearLayer(child, quantizer_class, exclude_list, quantized=quantized)

            

In [7]:
tokenzier = AutoTokenizer.from_pretrained("facebook/opt-350m")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", torch_dtype=torch.float16)

c:\Users\Anoop Maurya\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anoop Maurya\.cache\huggingface\hub\models--facebook--opt-350m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
